In [48]:
# pip install folium

In [49]:
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

# Step 3: Set up the API URL
api_url = 'https://data.nashville.gov/resource/qywv-8sc2.json'

# Step 4: Create a function to query the API and retrieve the JSON data
def query_api():
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print('Failed to query the API.')
        return None

# Step 5: Convert JSON data to a Pandas DataFrame
def json_to_dataframe(data):
    df = pd.DataFrame(data)
    return df

# Step 6: Save the DataFrame as a CSV file
def save_as_csv(df, filename):
    existing_data = pd.read_csv(filename)  # Read the existing CSV file
    combined_data = pd.concat([existing_data, df])  # Concatenate existing data with new data
    deduplicated_data = combined_data.drop_duplicates()  # Remove duplicates
    deduplicated_data.to_csv(filename, index=False)

# Step 7: Process addresses and convert them into points using Geopandas
def process_addresses(df):
    geocoded_data = []
    for index, row in df.iterrows():
        address = row['address']
        city = row['city']
        lat, lon = geocode_address(f'{address}, {city}, Tennessee')
        if lat and lon:
            row['latitude'] = lat
            row['longitude'] = lon
            geocoded_data.append(row)
    geocoded_df = pd.DataFrame(geocoded_data)
    return geocoded_df

# Function to geocode an address using the OpenStreetMap Nominatim API
def geocode_address(address):
    geocoding_url = f'https://nominatim.openstreetmap.org/search?format=json&q={address}'
    response = requests.get(geocoding_url)
    if response.status_code == 200:
        data = response.json()
        if data:
            lat = float(data[0]['lat'])
            lon = float(data[0]['lon'])
            return lat, lon
    return None, None

# Step 8: Plot the data on a map using Folium
def plot_on_map(df):
    m = folium.Map(location=[36.1612, -86.7775], zoom_start=11.3)

    # Add points to the map
    for index, row in df.iterrows():
        
        popup_text = f"Incident Type: {row['incident_type']}<br><br>Call Received: {row['call_received']}<br><br>Address: {row['address']}<br><br>City: {row['city']}"
        folium.Marker([row['latitude'], row['longitude']],
                      popup=popup_text).add_to(m)

    return m

# Re-read CS as DF
def reload_full_csv(filename):
    df = pd.read_csv(filename)  # Read the existing CSV file
    return df

# Step 1: Query the API and retrieve the JSON data
json_data = query_api()

# Step 2: Convert JSON data to a Pandas DataFrame
df_api = json_to_dataframe(json_data)

# Step 7: Process addresses and convert them into points using Geopandas
df = process_addresses(df_api)

# Step 6: Save the DataFrame as a CSV file
save_as_csv(df, 'data.csv')

# reload df
df_full = reload_full_csv('data.csv')

# Step 9: Plot the data on a map using Folium
map = plot_on_map(df_full)

# Step 9 (cont.): Customize the map to display different incident types as different colors
# Add code here to customize the map based on incident types

# Display the map
map.save('map.html')
